In [ ]:
pip install -r requirements.txt

In [ ]:
# computer
pip install python-dotenv

api_key = os.getenv('YOUTUBE_API_KEY')

In [ ]:
# for colab

from google.colab import userdata
api_key = userdata.get('YOUTUBE_API_KEY')

In [ ]:
import os
import re
import json
import datetime
from collections import Counter, defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
from dotenv import load_dotenv
from transformers import pipeline, AutoTokenizer
from googleapiclient.discovery import build
import isodate
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import networkx as nx

# NLTK veri setlerini indir
nltk.download('punkt')
nltk.download('stopwords')

# Çevresel değişkenleri yükle
load_dotenv()

# Duygu analizi için modeller
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

MAX_TOKENS = 512

def get_youtube_service():
    """YouTube API istemcisini başlat."""
    if not api_key:
        raise ValueError("Error: YouTube API key not found in environment variables")
    youtube = build('youtube', 'v3', developerKey=api_key)
    return youtube

def extract_video_id(youtube_url):
    """YouTube URL'sinden video ID'sini çıkar."""
    # Farklı URL formatlarını destekle
    regex_patterns = [
        r'(?:v=|\/)([0-9A-Za-z_-]{11}).*',
        r'(?:be\/)([0-9A-Za-z_-]{11}).*',
        r'(?:embed\/)([0-9A-Za-z_-]{11}).*'
    ]
    for pattern in regex_patterns:
        match = re.search(pattern, youtube_url)
        if match:
            return match.group(1)
    raise ValueError("Geçersiz YouTube URL'si. Lütfen doğru bir URL girin.")

def get_video_metadata(video_id, youtube):
    """Belirli bir video hakkında geniş metadata al."""
    response = youtube.videos().list(
        part="snippet,contentDetails,statistics,status,topicDetails,liveStreamingDetails",
        id=video_id
    ).execute()

    if not response['items']:
        raise ValueError("Bu ID ile eşleşen video bulunamadı.")
    
    video_data = response['items'][0]
    duration = isodate.parse_duration(video_data['contentDetails']['duration']).total_seconds()
    metadata = {
        'video_id': video_id,
        'title': video_data['snippet']['title'],
        'description': video_data['snippet']['description'],
        'tags': video_data['snippet'].get('tags', []),
        'channel_title': video_data['snippet']['channelTitle'],
        'channel_id': video_data['snippet']['channelId'],
        'publish_date': video_data['snippet']['publishedAt'],
        'thumbnail_url': video_data['snippet']['thumbnails']['high']['url'],
        'view_count': int(video_data['statistics'].get('viewCount', 0)),
        'like_count': int(video_data['statistics'].get('likeCount', 0)),
        'comment_count': int(video_data['statistics'].get('commentCount', 0)),
        'duration_seconds': duration,
        'licensed_content': video_data['contentDetails'].get('licensedContent', False),
        'privacy_status': video_data['status']['privacyStatus'],
        'embeddable': video_data['status']['embeddable'],
        'public_stats_viewable': video_data['status']['publicStatsViewable'],
        'category_id': video_data['snippet'].get('categoryId', None),
        'topic_categories': video_data.get('topicDetails', {}).get('topicCategories', []),
        'live_broadcast': video_data.get('liveStreamingDetails', {}).get('actualStartTime', None),
        'default_language': video_data['snippet'].get('defaultLanguage')
    }
    return metadata

def truncate_text(text, max_tokens=MAX_TOKENS):
    """Yorum metnini belirli bir token uzunluğuna kısaltır."""
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
    tokens = tokenizer.encode(text, truncation=True, max_length=max_tokens)
    return tokenizer.decode(tokens, skip_special_tokens=True)

def preprocess_text(text):
    """Metni ön işleme tabi tutar."""
    text = re.sub(r'http\S+', '', text)  # URL'leri kaldır
    text = re.sub(r'@\w+', '', text)     # Kullanıcı etiketlerini kaldır
    text = re.sub(r'#\w+', '', text)     # Hashtag'leri kaldır
    text = re.sub(r'\s+', ' ', text)     # Fazla boşlukları kaldır
    return text.strip()

def get_comment_sentiments(video_id, youtube, max_comments=1000):
    """Yorumları analiz ederek duygusal dağılım, anahtar kelimeler ve en aktif kullanıcıları bulur."""
    comments = []
    sentiment_counts = Counter()
    emotion_counts = Counter()
    active_users = Counter()
    key_phrases = Counter()
    comment_times = []
    comment_lengths = []
    comment_likes = []
    user_interactions = []
    
    stop_words = set(stopwords.words('english') + stopwords.words('turkish'))

    next_page_token = None

    print("Yorumlar indiriliyor ve analiz ediliyor...")
    pbar = tqdm(total=max_comments)
    while len(comments) < max_comments:
        response = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            textFormat="plainText",
            maxResults=min(max_comments - len(comments), 100),
            pageToken=next_page_token
        ).execute()

        for item in response.get('items', []):
            comment_snippet = item['snippet']['topLevelComment']['snippet']
            comment_text = comment_snippet['textDisplay']
            truncated_text = truncate_text(comment_text)
            preprocessed_text = preprocess_text(truncated_text)
            author = comment_snippet['authorDisplayName']
            like_count = comment_snippet['likeCount']
            published_at = comment_snippet['publishedAt']
            published_at_dt = datetime.datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ")
            comment_times.append(published_at_dt)
            comment_lengths.append(len(preprocessed_text.split()))
            comment_likes.append(like_count)

            # Duygu analizi
            sentiment = sentiment_analyzer(preprocessed_text)[0]
            sentiment_counts[sentiment['label']] += 1

            # Duygu yoğunluğu
            emotions = emotion_analyzer(preprocessed_text)[0]
            dominant_emotion = max(emotions, key=lambda x: x['score'])
            emotion_counts[dominant_emotion['label']] += 1

            # Anahtar kelime analizi
            words = word_tokenize(preprocessed_text)
            for word in words:
                word = word.lower()
                if word.isalpha() and word not in stop_words and len(word) > 3:
                    key_phrases[word] += 1

            # Aktif kullanıcılar
            active_users[author] += 1

            # Kullanıcı etkileşimleri (yanıtlar)
            if 'replies' in item:
                for reply in item['replies']['comments']:
                    reply_snippet = reply['snippet']
                    replier = reply_snippet['authorDisplayName']
                    user_interactions.append((author, replier))

            # Yorum kaydı
            comments.append({
                'comment_text': comment_text,
                'author': author,
                'like_count': like_count,
                'published_at': published_at,
                'sentiment': sentiment['label'],
                'emotion': dominant_emotion['label']
            })
            pbar.update(1)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    pbar.close()

    # En sık kullanılan 20 anahtar kelime
    top_keywords = key_phrases.most_common(20)
    # En aktif 10 kullanıcı
    top_active_users = active_users.most_common(10)

    return comments, dict(sentiment_counts), dict(emotion_counts), top_keywords, top_active_users, comment_times, comment_lengths, comment_likes, user_interactions

def generate_wordcloud(key_phrases):
    """Anahtar kelimelerin kelime bulutunu oluşturur."""
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(key_phrases))
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.savefig('wordcloud.png')
    plt.close()

def plot_sentiment_distribution(sentiment_counts):
    """Duygu dağılımını görselleştirir."""
    labels = list(sentiment_counts.keys())
    sizes = list(sentiment_counts.values())
    plt.figure(figsize=(10, 7))
    sns.barplot(x=labels, y=sizes)
    plt.title('Duygu Dağılımı')
    plt.xlabel('Duygu')
    plt.ylabel('Sayı')
    plt.savefig('sentiment_distribution.png')
    plt.close()

def plot_emotion_distribution(emotion_counts):
    """Duygu yoğunluğu dağılımını görselleştirir."""
    labels = list(emotion_counts.keys())
    sizes = list(emotion_counts.values())
    plt.figure(figsize=(12, 8))
    sns.barplot(x=labels, y=sizes)
    plt.title('Duygu Yoğunluğu Dağılımı')
    plt.xlabel('Duygu')
    plt.ylabel('Sayı')
    plt.savefig('emotion_distribution.png')
    plt.close()

def plot_comment_timeline(comment_times):
    """Yorumların zaman içindeki dağılımını görselleştirir."""
    df = pd.DataFrame({'timestamp': comment_times})
    df.set_index('timestamp', inplace=True)
    df['count'] = 1
    df = df.resample('D').sum()
    plt.figure(figsize=(15, 7))
    df['count'].plot()
    plt.title('Zaman İçinde Yorum Dağılımı')
    plt.xlabel('Tarih')
    plt.ylabel('Yorum Sayısı')
    plt.savefig('comment_timeline.png')
    plt.close()

def perform_topic_modeling(comments):
    """LDA ile konu modellemesi yapar."""
    print("Konu modellemesi yapılıyor...")
    texts = [preprocess_text(truncate_text(comment['comment_text'])) for comment in comments]
    stop_words = set(stopwords.words('english') + stopwords.words('turkish'))
    texts_tokenized = [[word for word in word_tokenize(text.lower()) if word.isalpha() and word not in stop_words] for text in texts]

    dictionary = corpora.Dictionary(texts_tokenized)
    corpus = [dictionary.doc2bow(text) for text in texts_tokenized]

    lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

    topics = lda_model.print_topics(num_words=5)
    topics_data = []
    for topic in topics:
        topics_data.append({'topic_id': topic[0], 'terms': topic[1]})

    # Sonuçları kaydet
    with open('topic_modeling.json', 'w', encoding='utf-8') as f:
        json.dump(topics_data, f, ensure_ascii=False, indent=4)
    print("Konu modelleme sonuçları 'topic_modeling.json' dosyasına kaydedildi.")

def analyze_top_comments(comments):
    """En çok beğenilen yorumları analiz eder."""
    print("En beğenilen yorumlar analiz ediliyor...")
    top_comments = sorted(comments, key=lambda x: x['like_count'], reverse=True)[:10]
    with open('top_comments.json', 'w', encoding='utf-8') as f:
        json.dump(top_comments, f, ensure_ascii=False, indent=4)
    print("En beğenilen yorumlar 'top_comments.json' dosyasına kaydedildi.")

def analyze_comment_lengths(comment_lengths):
    """Yorum uzunluğu istatistiklerini analiz eder."""
    print("Yorum uzunluğu istatistikleri analiz ediliyor...")
    average_length = sum(comment_lengths) / len(comment_lengths)
    median_length = sorted(comment_lengths)[len(comment_lengths) // 2]
    plt.figure(figsize=(10, 7))
    sns.histplot(comment_lengths, bins=30, kde=True)
    plt.title('Yorum Uzunluğu Dağılımı')
    plt.xlabel('Kelime Sayısı')
    plt.ylabel('Yorum Sayısı')
    plt.savefig('comment_length_distribution.png')
    plt.close()
    # Sonuçları kaydet
    length_stats = {
        'average_length': average_length,
        'median_length': median_length
    }
    with open('comment_length_stats.json', 'w', encoding='utf-8') as f:
        json.dump(length_stats, f, ensure_ascii=False, indent=4)
    print("Yorum uzunluğu istatistikleri 'comment_length_stats.json' dosyasına kaydedildi.")

def perform_clustering(comments):
    """Yorumları kümeleme analizi yapar."""
    print("Kümeleme analizi yapılıyor...")
    texts = [preprocess_text(truncate_text(comment['comment_text'])) for comment in comments]
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(texts)

    k = 5  # Küme sayısı
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    labels = kmeans.labels_

    # Sonuçları kaydet
    clustered_comments = defaultdict(list)
    for idx, label in enumerate(labels):
        clustered_comments[int(label)].append(comments[idx])

    with open('clustering_results.json', 'w', encoding='utf-8') as f:
        json.dump(clustered_comments, f, ensure_ascii=False, indent=4)
    print("Kümeleme sonuçları 'clustering_results.json' dosyasına kaydedildi.")

def analyze_user_interactions(user_interactions):
    """Kullanıcılar arasındaki etkileşimleri analiz eder."""
    print("Kullanıcı etkileşim ağı oluşturuluyor...")
    G = nx.DiGraph()
    G.add_edges_from(user_interactions)
    plt.figure(figsize=(15, 15))
    pos = nx.spring_layout(G, k=0.5)
    nx.draw(G, pos, node_size=50, with_labels=False)
    plt.savefig('user_interaction_network.png')
    plt.close()
    print("Kullanıcı etkileşim ağı 'user_interaction_network.png' olarak kaydedildi.")

def plot_sentiment_trends(comments):
    """Duygu eğilimlerini zaman içinde görselleştirir."""
    print("Duygu trendleri analiz ediliyor...")
    df = pd.DataFrame(comments)
    df['published_at'] = pd.to_datetime(df['published_at'])
    df.set_index('published_at', inplace=True)
    sentiment_trends = df.resample('D')['sentiment'].value_counts().unstack().fillna(0)
    sentiment_trends.plot(kind='line', figsize=(15, 7))
    plt.title('Zaman İçinde Duygu Trendleri')
    plt.xlabel('Tarih')
    plt.ylabel('Yorum Sayısı')
    plt.savefig('sentiment_trends.png')
    plt.close()
    print("Duygu trend grafiği 'sentiment_trends.png' olarak kaydedildi.")

def save_metadata_to_json(metadata):
    """Video metadata bilgilerini JSON dosyasına kaydeder."""
    with open('video_metadata.json', 'w', encoding='utf-8') as json_file:
        json.dump(metadata, json_file, ensure_ascii=False, indent=4)

def save_sentiment_analysis_to_json(comments, sentiment_counts, emotion_counts, top_keywords, top_active_users):
    """Sentiment analiz sonuçlarını JSON dosyasına kaydeder."""
    data = {
        'comments': comments,
        'sentiment_distribution': sentiment_counts,
        'emotion_distribution': emotion_counts,
        'top_keywords': top_keywords,
        'top_active_users': top_active_users
    }
    with open('sentiment_analysis.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

def main():
    youtube_url = input("YouTube Video URL'sini girin: ")
    youtube = get_youtube_service()
    video_id = extract_video_id(youtube_url)
    
    # 1. Video Metadata Analizi
    print("Video metadata bilgileri alınıyor...")
    metadata = get_video_metadata(video_id, youtube)
    save_metadata_to_json(metadata)
    print("Video metadata bilgileri 'video_metadata.json' dosyasına kaydedildi.")

    # 2. Sentiment Analizi ve Ekstra Analizler
    comments, sentiment_counts, emotion_counts, top_keywords, top_active_users, comment_times, comment_lengths, comment_likes, user_interactions = get_comment_sentiments(video_id, youtube, max_comments=1000)
    save_sentiment_analysis_to_json(comments, sentiment_counts, emotion_counts, top_keywords, top_active_users)
    print("Sentiment analiz sonuçları 'sentiment_analysis.json' dosyasına kaydedildi.")

    # 3. Görselleştirmeler ve Ek Analizler
    generate_wordcloud(top_keywords)
    print("Kelime bulutu 'wordcloud.png' olarak kaydedildi.")

    plot_sentiment_distribution(sentiment_counts)
    print("Duygu dağılım grafiği 'sentiment_distribution.png' olarak kaydedildi.")

    plot_emotion_distribution(emotion_counts)
    print("Duygu yoğunluğu grafiği 'emotion_distribution.png' olarak kaydedildi.")

    plot_comment_timeline(comment_times)
    print("Yorum zaman çizelgesi 'comment_timeline.png' olarak kaydedildi.")

    perform_topic_modeling(comments)

    analyze_top_comments(comments)

    analyze_comment_lengths(comment_lengths)

    perform_clustering(comments)

    analyze_user_interactions(user_interactions)

    plot_sentiment_trends(comments)

if __name__ == "__main__":
    main()
